In [128]:
import pandas as pd
from pykml import parser
from os import path
import numpy as np

def read_kml(file_path):
    with open(file_path) as f:
        doc = parser.parse(f).getroot()
    return doc

def get_placemarks(root):
    return root.Document.Folder.Placemark

def get_line_strings(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}LineString')

kml_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.kml'
csv_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.csv'
csv_file_crash = "/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/Crash_Data_53640.csv"

# Read pavement and crash data
doc = read_kml(kml_file_pavement)
placemarks_pavement = get_placemarks(doc)
df_crash = pd.read_csv(csv_file_crash)

linestring_list = []
for placemark in placemarks_pavement:
    line_strings = get_line_strings(placemark)
    for line_string in line_strings:
        coords = line_string.coordinates.text.strip().split(' ')
        coords = [(float(coord.split(',')[0]), float(coord.split(',')[1])) for coord in coords if coord]
        linestring_list.append(coords)

df_pavement = pd.read_csv(csv_file_pavement)
df_pavement['Crashes'] = 0

# Extract X and Y columns for efficiency
df_crash_xy = df_crash[['X', 'Y']].values

# Define a tolerance threshold
tolerance = 0.00015

# Corrected loop for crash counting with tolerance
for i, line_string in enumerate(linestring_list):
    crashes = 0
    for point in line_string:
        x_diff = np.abs(df_crash_xy[:, 0] - point[0])
        y_diff = np.abs(df_crash_xy[:, 1] - point[1])
        crashes += np.sum((x_diff <= 0.0001) & (y_diff <= 0.0001))
    df_pavement.loc[i, 'Crashes'] = crashes
    print(i + 1, crashes)


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_11302/3665748688.py:34: DtypeWarning: Columns (11,87,99,115,116,119,120,121,122,123,131,132,147,148,150,151,152,153,154,155,156,185,186,187,188,192,193,194,195,196,199,200,201,202,209,210,211,212,215,218,219,225) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pavement = pd.read_csv(csv_file_pavement)


1 36
2 9
3 19
4 0
5 5
6 6
7 3
8 3
9 1
10 0
11 0
12 1
13 0
14 2
15 0
16 1
17 0
18 0
19 2
20 0
21 2
22 0
23 0
24 3
25 2
26 0
27 0
28 0
29 0
30 0
31 0
32 1
33 0
34 0
35 4
36 1
37 4
38 4
39 3
40 0
41 3
42 0
43 11
44 4
45 20
46 1
47 7
48 16
49 28
50 14
51 10
52 0
53 2
54 0
55 0
56 0
57 2
58 1
59 0
60 3
61 1
62 0
63 3
64 0
65 0
66 1
67 0
68 0
69 0
70 4
71 0
72 5
73 2
74 1
75 2
76 2
77 0
78 1
79 0
80 1
81 1
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 1
91 0
92 0
93 0
94 1
95 0
96 0
97 0
98 1
99 2
100 0
101 0
102 1
103 0
104 2
105 0
106 2
107 1


KeyboardInterrupt: 

In [ ]:
df_pavement["Crashes"].sum()

17799.0

In [ ]:
import pandas as pd
from pykml import parser
import numpy as np
import xml.etree.ElementTree as ET

def read_kml(file_path):
    with open(file_path) as f:
        doc = parser.parse(f).getroot()
    return doc

def get_placemarks(root):
    return root.Document.Folder.Placemark

def get_line_strings(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}LineString')

def get_origkey(placemark):
    # Adjusted to correctly navigate and extract ORIGKEY
    if hasattr(placemark, 'ExtendedData'):
        schema_data = placemark.ExtendedData.find('.//{http://www.opengis.net/kml/2.2}SchemaData')
        if schema_data is not None:
            for simple_data in schema_data.findall('.//{http://www.opengis.net/kml/2.2}SimpleData'):
                if simple_data.attrib.get('name') == 'ORIGKEY':
                    return simple_data.text
    return 'Unknown'

# File paths
kml_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.kml'
csv_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.csv'
csv_file_crash = "/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/Crash_Data_53640.csv"

# Read pavement and crash data
doc = read_kml(kml_file_pavement)
placemarks_pavement = get_placemarks(doc)
df_crash = pd.read_csv(csv_file_crash)

linestring_list = []
origkey_list = []

# Extract linestrings and ORIGKEYs
for placemark in placemarks_pavement:
    origkey = get_origkey(placemark)
    line_strings = get_line_strings(placemark)
    for line_string in line_strings:
        coords = line_string.coordinates.text.strip().split(' ')
        coords = [(float(coord.split(',')[0]), float(coord.split(',')[1])) for coord in coords if coord]
        linestring_list.append(coords)
        origkey_list.append(origkey)

# Define a tolerance threshold
tolerance = 0.00015

# Create a list to store matched pavement data
matched_pavement_data = []

# Match crashes to pavements
for i, line_string in enumerate(linestring_list):
    matched_crashes = []
    for point in line_string:
        x_diff = np.abs(df_crash['X'] - point[0])
        y_diff = np.abs(df_crash['Y'] - point[1])
        matched_crashes.extend(df_crash[(x_diff <= 0.0001) & (y_diff <= 0.0001)].values.tolist())

    if len(matched_crashes) > 0:
        origkey = origkey_list[i]
        for crash_data in matched_crashes:
            matched_pavement_data.append([origkey] + crash_data)

# Create a DataFrame for matched pavement data
pavement_columns = df_crash.columns.tolist()
pavement_columns.insert(0, 'Pavement_ORIGKEY')
df_matched_pavement = pd.DataFrame(matched_pavement_data, columns=pavement_columns)

# You can then process df_matched_pavement as needed


In [ ]:
df_matched_pavement.to_csv('/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/Test_Pavement_new_originkey.csv')

In [ ]:
import pandas as pd
from pykml import parser
import numpy as np

def read_kml(file_path):
    with open(file_path) as f:
        doc = parser.parse(f).getroot()
    return doc

def get_placemarks(root):
    return root.Document.Folder.Placemark

def get_line_strings(placemark):
    return placemark.findall('.//{http://www.opengis.net/kml/2.2}LineString')

def get_extended_data(placemark):
    # Retrieve all extended data attributes as a dictionary
    data = {}
    if hasattr(placemark, 'ExtendedData'):
        schema_data = placemark.ExtendedData.find('.//{http://www.opengis.net/kml/2.2}SchemaData')
        if schema_data is not None:
            for simple_data in schema_data.findall('.//{http://www.opengis.net/kml/2.2}SimpleData'):
                data[simple_data.attrib['name']] = simple_data.text
    return data

# File paths
kml_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.kml'
csv_file_pavement = '/Users/lebakuprathyushkumarreddy/Pavement.csv'
csv_file_crash = "/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/Crash_Data_53640.csv"

# Read pavement and crash data
doc = read_kml(kml_file_pavement)
placemarks_pavement = get_placemarks(doc)
df_crash = pd.read_csv(csv_file_crash)

pavement_data = []

# Extract linestrings and attributes
for placemark in placemarks_pavement:
    extended_data = get_extended_data(placemark)
    line_strings = get_line_strings(placemark)
    for line_string in line_strings:
        coords = line_string.coordinates.text.strip().split(' ')
        coords = [(float(coord.split(',')[0]), float(coord.split(',')[1])) for coord in coords if coord]
        # Append coordinates and extended data to the list
        pavement_data.append((coords, extended_data))

# Define a tolerance threshold
tolerance = 0.00015  # You can adjust this tolerance based on your data

# Create a list to store matched pavement data
matched_pavement_data = []

# Match crashes to pavements
for coords, extended_data in pavement_data:
    for point in coords:
        x_diff = np.abs(df_crash['X'] - point[0])
        y_diff = np.abs(df_crash['Y'] - point[1])
        matched_crashes = df_crash[(x_diff <= tolerance) & (y_diff <= tolerance)]
        for _, crash_data in matched_crashes.iterrows():
            # Merge the attributes from KML and crash data
            merged_data = {**crash_data.to_dict(),**extended_data}
            matched_pavement_data.append(merged_data)

# Create a DataFrame for matched pavement data
df_matched_pavement = pd.DataFrame(matched_pavement_data)

# Now df_matched_pavement contains all the attributes from both KML and CSV


KeyboardInterrupt: 

In [ ]:
df_matched_pavement["Crashes"].sum()

KeyError: 'Crashes'

In [ ]:
df_pavement["Crashes"].sum()

23882.0

In [282]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from bs4 import BeautifulSoup
from IPython.display import display

# Step 1: Read the CSV and KML files
csv_file_path = "/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/Crash_Data_53640.csv"
kml_file_path = '/Users/lebakuprathyushkumarreddy/Pavement.kml'

# Read the CSV file and create a GeoDataFrame
csv_data = pd.read_csv(csv_file_path)
geometry = [Point(lon, lat) for lon, lat in zip(csv_data['X'], csv_data['Y'])]
csv_gdf = gpd.GeoDataFrame(csv_data, geometry=geometry, crs='EPSG:4326')  # Use the appropriate CRS for your data

# Parse the KML file with BeautifulSoup
with open(kml_file_path, 'r') as kml_file:
    kml_content = kml_file.read()

soup = BeautifulSoup(kml_content, 'xml')

# Extract LineString geometries from KML
line_geometries = []
iri_values = []  # List to store IRI values
origkey_values=[]

attributes =["OBJECTID", "ORIGKEY", "PMISYR", "ROUTE_ID", "FROM_MEASURE", "TO_MEASURE", "SYSTEM", 
    "ROUTE", "DIR", "BPOST", "EPOST", "PAVTYP", "DESCRIPT", "CONYR", "RESYR", "PMIS_LENGTH", 
    "LANE_MILES", "MDIST", "COUNTY", "CITY", "PCI_2", "PCI_2DEF", "RUT_INDX", "IRI_INDX", 
    "FAULT_INDX", "CRACK_INDX", "IRI", "FRICT", "FRIDATE", "FAULT", "FAULTAV", "RUT", 
    "CRACK_RATIO", "T_INDX", "L_INDX", "LW_INDX", "LLW_INDX", "A_INDX", "ACRACKH", "ACRACKM", 
    "ACRACKL", "TCRACKH", "TCRACKM", "TCRACKL", "LCRACKH", "LCRACKM", "LCRACKL", "LCRACKWH", 
    "LCRACKWM", "LCRACKWL", "DCRACKH", "DCRACKM", "DCRACKL", "JTSPALLH", "JTSPALLM", 
    "JTSPALL_LONG", "JTSPALL_TRANS", "PATCHAB", "PATCHAG", "PATCHES", "COVERAGE", "CAPDAT", 
    "STRUC80", "STRUCT_NEED80", "STRUCAV", "STRUCJTR", "AVEK", "FWD_DATE", "TRYR", "AADT", 
    "TRUCKS", "NHS", "SYS_NHS", "URBAN", "PCLASS", "FCLASS", "MEDIAN", "MEDIAN_PCT", "LANES", 
    "SPEED", "SURFACE_TYPE", "PAVTHICK", "TACCDPTH", "TPCCDPTH", "BASEDPTH", "MLEVEL", "WIDTH", 
    "WDRIVELN", "RCURB", "LCURB", "ISHLDTIE", "ISHLDTYP", "ISHLDWID", "OSHLDTIE", "OSHLDTYP", 
    "OSHLDWID", "COMPLEX", "SUBDRAIN", "SUBDPCT", "SUBDMULT", "TREATMENT", "LAYR1", "LAYR2", 
    "LAYR3", "LAYR4", "LAYR5", "LAYR6", "LAYR7", "LAYR8", "PROJECT1", "PROJECT2", "PROJECT3", 
    "PROJECT4", "PROJECT5", "PROJECT6", "PROJECT7", "PROJECT8", "PROJTYP1", "PROJTYP2", 
    "PROJTYP3", "PROJTYP4", "PROJTYP5", "PROJTYP6", "PROJTYP7", "PROJTYP8", "SURTYP1", "SURTYP2", 
    "SURTYP3", "SURTYP4", "SURTYP5", "SURTYP6", "SURTYP7", "SURTYP8", "SURTHK1", "SURTHK2", 
    "SURTHK3", "SURTHK4", "SURTHK5", "SURTHK6", "SURTHK7", "SURTHK8", "BASTYP1", "BASTYP2", 
    "BASTYP3", "BASTYP4", "BASTYP5", "BASTYP6", "BASTYP7", "BASTYP8", "AGGCLAS1", "AGGCLAS2", 
    "AGGCLAS3", "AGGCLAS4", "AGGCLAS5", "AGGCLAS6", "AGGCLAS7", "AGGCLAS8", "BASTHK1", "BASTHK2", 
    "BASTHK3", "BASTHK4", "BASTHK5", "BASTHK6", "BASTHK7", "BASTHK8", "SUBTHK1", "SUBTHK2", 
    "SUBTHK3", "SUBTHK4", "SUBTHK5", "SUBTHK6", "SUBTHK7", "SUBTHK8", "RMVTHK1", "RMVTHK2", 
    "RMVTHK3", "RMVTHK4", "RMVTHK5", "RMVTHK6", "RMVTHK7", "RMVTHK8", "AGGSRC1", "AGGSRC2", 
    "AGGSRC3", "AGGSRC4", "AGGSRC5", "AGGSRC6", "AGGSRC7", "AGGSRC8", "REMARKS1", "REMARKS2", 
    "REMARKS3", "REMARKS4", "REMARKS5", "REMARKS6", "REMARKS7", "REMARKS8", "RMVTYP1", "RMVTYP2", 
    "RMVTYP3", "RMVTYP4", "RMVTYP5", "RMVTYP6", "RMVTYP7", "RMVTYP8", "AGGTYP1", "AGGTYP2", 
    "AGGTYP3", "AGGTYP4", "AGGTYP5", "AGGTYP6", "AGGTYP7", "AGGTYP8", "SUBTYP1", "SUBTYP2", 
    "SUBTYP3", "SUBTYP4", "SUBTYP5", "SUBTYP6", "SUBTYP7", "SUBTYP8", "REST_UPDATED", 
    "REST_UTC_OFFSET", "GLOBALID", "PATCH_PCT", "CONRTE", "FAILURE_AREA", "FAILURE_AREA_MI", 
    "FAILURE_CNT", "FAILURE_CNT_MI", "PATHWEB", "STRUC_C_PCT", "UNSEALCRK", "Shape__Length"]

# Initialize a dictionary to hold lists of values for each attribute
values_dict = {attribute: [] for attribute in attributes}
line_geometries = []

placemarks = soup.find_all('Placemark')
for placemark in placemarks:
    line = placemark.find('LineString')
    coordinates = line.find('coordinates').text.strip()
    coordinates = [tuple(map(float, coord.split(','))) for coord in coordinates.split(',0') if coord!='']
    line_geometry = LineString(coordinates)
    line_geometries.append(line_geometry)

    for attribute in attributes:
        simpledata = placemark.find('SimpleData', {'name': attribute})
        value = simpledata.text.strip() if simpledata else None
        values_dict[attribute].append(value)

# Create a GeoDataFrame for KML LineStrings
kml_gdf = gpd.GeoDataFrame(geometry=line_geometries, crs='EPSG:4326')  # Use the appropriate CRS for your data

# Add IRI values as a column to the GeoDataFrame
for attribute in values_dict:
    kml_gdf[attribute] = values_dict[attribute]
# Buffer the lines to create a buffer around them
buffer_distance = 0.0001  # Adjust this distance as needed (in degrees)
kml_gdf['buffered_geometry'] = kml_gdf['geometry'].buffer(buffer_distance)

# # Set the active geometry column for csv_gdf
csv_gdf = csv_gdf.set_geometry('geometry')
kml_gdf = kml_gdf.set_geometry('buffered_geometry')

# Perform spatial analysis to find points close to buffered lines
points_near_lines = gpd.sjoin(csv_gdf, kml_gdf, op='intersects')

# Print the result or perform further analysis
# print("Number of data points located near KML lines:", len(points_near_lines))


/Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/Users/lebakuprathyushkumarreddy/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [303]:
points_near_lines["IRI"].count()

21461

In [324]:
import pandas as pd

# Read the CSV file
csv_file_path = "/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv"
df = pd.read_csv("/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv")

df.fillna(0,inplace=True)

data= df[df["CRCOMNNR"]==1]


# Group by 'OBJECTID' and count the crashes
crash_counts = data.groupby('OBJECTID').size().reset_index(name='Crashes')
crash_counts
# # Aggregate other attributes per 'OBJECTID' (example: mean for numerical columns)
# # Adjust this based on how you want to aggregate other columns
aggregated_data = data.groupby('OBJECTID').mean()
aggregated_data

# Create a DataFrame with all OBJECTID values from 1 to 4119
all_objectids = pd.DataFrame({'OBJECTID': range(1, 4120)})

# Merge the all_objectids DataFrame with the aggregated data
# This ensures all 4119 segments are included
final_data = all_objectids.merge(aggregated_data, on='OBJECTID', how='left')

# Merge the crash counts with the final data
final_data = final_data.merge(crash_counts, on='OBJECTID', how='left')

# Fill NaN values in 'Crashes' with 0
final_data['Crashes'] = final_data['Crashes'].fillna(0).astype(int)

final_data["Crash_Rate"]= (final_data['Crashes']*100000000)/(final_data["AADT"]*final_data["PMIS_LENGTH"]*365)

final_data


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_11302/975452244.py:5: DtypeWarning: Columns (143,172,176,177,178,179,188,203,204,207,208,209,210,211,212,243,244,252,257,258,267,268,274,275,281) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv")
/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_11302/975452244.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby('OBJECTID').mean()


,OBJECTID,Unnamed: 0,X,Y,Crash_OBJECTID,CRASH_KEY,CASENUMBER,CRASH_MONTH,CRASH_DAY,DISTRICT,...,PATCH_PCT,FAILURE_AREA,FAILURE_AREA_MI,FAILURE_CNT,FAILURE_CNT_MI,STRUC_C_PCT,UNSEALCRK,Shape__Length,Crashes,Crash_Rate
0,1,26440.454545,-91.534385,41.657868,1.317491e+07,2.022131e+10,2.022131e+10,6.818182,3.727273,6.0,...,2.0,32.0,43.8,10.0,13.7,6.0,100.0,1572.384335,11,262.400959
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,3,51392.000000,-91.522825,41.671966,1.319986e+07,2.022134e+10,2.022134e+10,12.000000,6.000000,6.0,...,0.0,0.0,0.0,0.0,0.0,9.0,73.0,2082.699791,1,45.299703
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,5,7158.666667,-91.512625,41.678985,1.315563e+07,2.022129e+10,2.022129e+10,2.333333,4.000000,6.0,...,0.0,38.0,26.9,8.0,5.7,3.0,100.0,3049.606515,3,32.656607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4114,4115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4115,4116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4116,4117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4117,4118,26097.000000,-94.378942,42.267053,1.317457e+07,2.022130e+10,2.022130e+10,6.000000,1.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,13054.969770,1,40.774708
